<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[Вариант задания 10
Описание проекта:
Создать базовый класс Invoice в C#, который будет представлять информацию о фактурах за поставленные товары или оказанные услуги. На основе этого класса разработать 2-3 производных класса, демонстрирующих принципы наследования и полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и методы, а также переопределены некоторые методы базового класса для демонстрации полиморфизма.
Требования к базовому классу Invoice:

• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая сумма (TotalAmount). • Методы: o o CalculateTotal(): метод для расчета общей суммы по фактуре. o AddLine(LineItem lineItem): метод для добавления позиции в фактуру. o RemoveLine(LineItem lineItem): метод для удаления позиции из фактуры.

Требования к производным классам:

ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен быть переопределен для добавления информации о дате поставки товара при добавлении позиции.
УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные атрибуты, такие как Дата оказания услуги (ServiceDate). Метод RemoveLine() должен быть переопределен для добавления информации о причине аннулирования услуги при удалении позиции.
КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс): Должна содержать дополнительные атрибуты, такие как Наличие возврата (ReturnAllowed). Метод CalculateTotal() должен быть переопределен для учета возможного возврата товара или услуги при расчете общей суммы.]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [4]:
using System;
using System.Collections.Generic;
using System.Linq;

public delegate void InvoiceEventHandler(object sender, InvoiceEventArgs e);
public delegate void LineItemEventHandler(object sender, LineItemEventArgs e);
public delegate void PaymentProcessedHandler(string invoiceNumber, decimal amount);

public class InvoiceEventArgs : EventArgs
{
    public string InvoiceNumber { get; set; }
    public string Message { get; set; }
    public DateTime EventDate { get; set; }
    
    public InvoiceEventArgs(string invoiceNumber, string message)
    {
        InvoiceNumber = invoiceNumber;
        Message = message;
        EventDate = DateTime.Now;
    }
}

public class LineItemEventArgs : EventArgs
{
    public string Description { get; set; }
    public decimal Amount { get; set; }
    public string Action { get; set; }
    
    public LineItemEventArgs(string description, decimal amount, string action)
    {
        Description = description;
        Amount = amount;
        Action = action;
    }
}

public class InvoiceCollection<T> where T : Invoice
{
    private List<T> invoices = new List<T>();

    public event Action<string, string> CollectionChanged;

    public void AddInvoice(T invoice)
    {
        invoices.Add(invoice);
        Console.WriteLine($"Фактура {invoice.InvoiceNumber} добавлена");
        CollectionChanged?.Invoke(invoice.InvoiceNumber, "Добавлена в коллекцию");
    }

    public void RemoveInvoice(T invoice)
    {
        invoices.Remove(invoice);
        Console.WriteLine($"Фактура {invoice.InvoiceNumber} удалена");
        CollectionChanged?.Invoke(invoice.InvoiceNumber, "Удалена из коллекции");
    }

    public void PrintAllInvoices()
    {
        Console.WriteLine($"\nВсе фактуры ({invoices.Count} шт.):");
        foreach (var invoice in invoices)
        {
            invoice.PrintInfo();
        }
    }

    public T FindInvoiceByNumber(string invoiceNumber)
    {
        return invoices.FirstOrDefault(i => i.InvoiceNumber == invoiceNumber);
    }

    public IEnumerable<T> GetInvoicesByDateRange(DateTime startDate, DateTime endDate)
    {
        return invoices.Where(i => i.IssueDate >= startDate && i.IssueDate <= endDate);
    }

    public decimal GetTotalAmount()
    {
        return invoices.Sum(i => i.TotalAmount);
    }

    public int GetInvoiceCount()
    {
        return invoices.Count;
    }
}

public class LineItem
{
    public string Description { get; set; }
    public decimal UnitPrice { get; set; }
    public int Quantity { get; set; }
    public decimal Amount => UnitPrice * Quantity;

    public string Category { get; set; }
    public string UnitOfMeasure { get; set; }
    public decimal Discount { get; set; }
    public int LineNumber { get; set; }

    public LineItem(string description, decimal unitPrice, int quantity)
    {
        Description = description;
        UnitPrice = unitPrice;
        Quantity = quantity;
        Category = "Общая";
        UnitOfMeasure = "шт.";
        Discount = 0;
        LineNumber = 0;
    }

    public void ApplyDiscount(decimal discountAmount)
    {
        Discount = discountAmount;
        Console.WriteLine($"Скидка на позицию: {discountAmount} руб.");
    }

    public void ApplyPercentageDiscount(double percentage)
    {
        Discount = Amount * (decimal)(percentage / 100);
        Console.WriteLine($"Скидка {percentage}% на позицию: {Discount} руб.");
    }

    public decimal GetNetAmount()
    {
        return Amount - Discount;
    }

    public void ChangeUnitOfMeasure(string newUnit)
    {
        UnitOfMeasure = newUnit;
        Console.WriteLine($"Единица измерения изменена на: {newUnit}");
    }
}

public class Invoice
{
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; protected set; }
    protected List<LineItem> LineItems { get; set; }

    private string paymentMethod;
    protected string currency;
    internal string paymentTerms;
    public string invoiceType;

    private string invoiceStatus;
    protected string customerName;
    internal string billingAddress;
    public DateTime dueDate;

    public event InvoiceEventHandler InvoiceCreated;
    public event InvoiceEventHandler InvoiceUpdated;
    public event LineItemEventHandler LineItemAdded;
    public event LineItemEventHandler LineItemRemoved;

    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        LineItems = new List<LineItem>();
        TotalAmount = 0;
        
        paymentMethod = "Безналичный";
        currency = "RUB";
        paymentTerms = "30 дней";
        invoiceType = "Обычная";

        invoiceStatus = "Черновик";
        customerName = "Клиент";
        billingAddress = "Адрес";
        dueDate = issueDate.AddDays(30);

        OnInvoiceCreated("Фактура создана");
    }

    protected virtual void OnInvoiceCreated(string message)
    {
        InvoiceCreated?.Invoke(this, new InvoiceEventArgs(InvoiceNumber, message));
    }

    protected virtual void OnInvoiceUpdated(string message)
    {
        InvoiceUpdated?.Invoke(this, new InvoiceEventArgs(InvoiceNumber, message));
    }

    protected virtual void OnLineItemAdded(string description, decimal amount)
    {
        LineItemAdded?.Invoke(this, new LineItemEventArgs(description, amount, "Добавлена"));
    }

    protected virtual void OnLineItemRemoved(string description, decimal amount)
    {
        LineItemRemoved?.Invoke(this, new LineItemEventArgs(description, amount, "Удалена"));
    }

    public void ChangeStatus(string newStatus)
    {
        string oldStatus = invoiceStatus;
        invoiceStatus = newStatus;
        Console.WriteLine($"Статус изменен: {oldStatus} -> {newStatus}");
        OnInvoiceUpdated($"Статус изменен на {newStatus}");
    }

    public void SetCustomer(string name, string address)
    {
        customerName = name;
        billingAddress = address;
        Console.WriteLine($"Клиент: {name}, Адрес: {address}");
    }

    public void ExtendDueDate(int additionalDays)
    {
        dueDate = dueDate.AddDays(additionalDays);
        Console.WriteLine($"Новая дата оплаты: {dueDate:dd.MM.yyyy}");
    }

    public void SendReminder()
    {
        if (DateTime.Now > dueDate)
        {
            Console.WriteLine($"Напоминание: Фактура {InvoiceNumber} просрочена!");
        }
        else
        {
            Console.WriteLine($"Напоминание: Фактура {InvoiceNumber} до {dueDate:dd.MM.yyyy}");
        }
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = LineItems.Sum(item => item.Amount);
    }

    public virtual void AddLine(LineItem lineItem)
    {
        LineItems.Add(lineItem);
        CalculateTotal();
        OnLineItemAdded(lineItem.Description, lineItem.Amount);
    }

    public virtual void AddLine(string description, decimal unitPrice, int quantity)
    {
        var lineItem = new LineItem(description, unitPrice, quantity);
        LineItems.Add(lineItem);
        CalculateTotal();
        Console.WriteLine($"Добавлено: {description}");
        OnLineItemAdded(description, lineItem.Amount);
    }

    public virtual void RemoveLine(LineItem lineItem)
    {
        LineItems.Remove(lineItem);
        CalculateTotal();
        OnLineItemRemoved(lineItem.Description, lineItem.Amount);
    }

    public string GetPaymentMethod() => paymentMethod;
    
    public void ChangeCurrency(string newCurrency)
    {
        currency = newCurrency;
        Console.WriteLine($"Валюта: {newCurrency}");
        OnInvoiceUpdated($"Валюта изменена на {newCurrency}");
    }
    
    public void ChangeCurrency(string newCurrency, decimal exchangeRate)
    {
        currency = newCurrency;
        TotalAmount *= exchangeRate;
        Console.WriteLine($"Валюта: {newCurrency}, курс: {exchangeRate}");
        OnInvoiceUpdated($"Валюта изменена на {newCurrency} с курсом {exchangeRate}");
    }

    public virtual void ApplyDiscount(decimal discount)
    {
        TotalAmount -= discount;
        if (TotalAmount < 0) TotalAmount = 0;
        Console.WriteLine($"Скидка: {discount} руб.");
    }

    public virtual void ApplyDiscountPercent(double percentage)
    {
        decimal discountAmount = TotalAmount * (decimal)(percentage / 100);
        TotalAmount -= discountAmount;
        Console.WriteLine($"Скидка {percentage}%: -{discountAmount} руб.");
    }

    public void ValidateInvoice()
    {
        Console.WriteLine($"Фактура {InvoiceNumber} проверена");
        ChangeStatus("Проверена");
    }

    public virtual void PrintInfo()
    {
        Console.WriteLine($"Фактура #{InvoiceNumber}");
        Console.WriteLine($"Дата: {IssueDate:dd.MM.yyyy}");
        Console.WriteLine($"Сумма: {TotalAmount} {currency}");
    }

    public virtual void PrintInfo(bool detailed)
    {
        if (detailed)
        {
            Console.WriteLine($"--- Фактура #{InvoiceNumber} ---");
            Console.WriteLine($"Дата: {IssueDate:dd.MM.yyyy}");
            Console.WriteLine($"Тип: {invoiceType}");
            Console.WriteLine($"Сумма: {TotalAmount} {currency}");
            Console.WriteLine($"Оплата: {paymentMethod}");
            Console.WriteLine($"Условия: {paymentTerms}");
            Console.WriteLine($"Позиций: {LineItems.Count}");
            Console.WriteLine($"Статус: {invoiceStatus}");
            Console.WriteLine($"Клиент: {customerName}");
            Console.WriteLine($"Оплата до: {dueDate:dd.MM.yyyy}");
        }
        else
        {
            PrintInfo();
        }
    }
}

public class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set; }

    private string supplierName;
    protected string deliveryMethod;
    internal string warehouseLocation;
    public bool isFragile;

    private string deliveryStatus;
    protected string trackingNumber;
    internal decimal shippingCost;
    public bool requiresInsurance;

    public event Action<string, DateTime> DeliveryScheduled;

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate)
        : base(invoiceNumber, issueDate)
    {
        SupplyDate = supplyDate;
        
        supplierName = "Поставщик";
        deliveryMethod = "Курьер";
        warehouseLocation = "Склад";
        isFragile = false;
        invoiceType = "Товары";

        deliveryStatus = "Запланирована";
        trackingNumber = "TRK-" + invoiceNumber;
        shippingCost = 0;
        requiresInsurance = false;
    }

    public void UpdateDeliveryStatus(string status)
    {
        deliveryStatus = status;
        Console.WriteLine($"Статус доставки: {status}");
        OnInvoiceUpdated($"Статус доставки изменен на {status}");
    }

    public void SetTrackingNumber(string tracking)
    {
        trackingNumber = tracking;
        Console.WriteLine($"Трек номер: {tracking}");
    }

    public void AddShippingCost(decimal cost)
    {
        shippingCost = cost;
        TotalAmount += cost;
        Console.WriteLine($"Стоимость доставки: {cost} руб.");
    }

    public void RequireInsurance(bool require)
    {
        requiresInsurance = require;
        if (require)
        {
            Console.WriteLine("Требуется страховка доставки");
        }
    }

    public string GetSupplierName() => supplierName;
    
    public void MarkAsFragile()
    {
        isFragile = true;
        Console.WriteLine("Товар хрупкий");
    }
    
    public void MarkAsFragile(string level)
    {
        isFragile = true;
        Console.WriteLine($"Товар хрупкий (уровень: {level})");
    }

    public void ScheduleDelivery(DateTime date)
    {
        SupplyDate = date;
        Console.WriteLine($"Доставка: {date:dd.MM.yyyy}");
        DeliveryScheduled?.Invoke(InvoiceNumber, date);
    }
    
    public void ScheduleDelivery(DateTime date, string deliveryType)
    {
        SupplyDate = date;
        deliveryMethod = deliveryType;
        Console.WriteLine($"Доставка {deliveryType}: {date:dd.MM.yyyy}");
        DeliveryScheduled?.Invoke(InvoiceNumber, date);
    }

    public void CheckStock()
    {
        Console.WriteLine($"Проверка склада {warehouseLocation}");
    }

    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Товар '{lineItem.Description}' поставка {SupplyDate:dd.MM.yyyy}");
    }

    public override void AddLine(string description, decimal unitPrice, int quantity)
    {
        var lineItem = new LineItem(description, unitPrice, quantity);
        base.AddLine(lineItem);
        Console.WriteLine($"Товар '{description}' поставка {SupplyDate:dd.MM.yyyy}");
    }

    public override void PrintInfo()
    {
        base.PrintInfo();
        Console.WriteLine($"Поставка: {SupplyDate:dd.MM.yyyy}");
        Console.WriteLine($"Поставщик: {supplierName}");
        Console.WriteLine($"Статус доставки: {deliveryStatus}");
        if (!string.IsNullOrEmpty(trackingNumber))
        {
            Console.WriteLine($"Трек номер: {trackingNumber}");
        }
    }

    public override void ApplyDiscount(decimal discount)
    {
        decimal maxDiscount = TotalAmount * 0.3m;
        decimal actualDiscount = discount > maxDiscount ? maxDiscount : discount;
        base.ApplyDiscount(actualDiscount);
        Console.WriteLine($"Скидка на товары: {actualDiscount} руб. (макс 30%)");
    }

    public override void ApplyDiscountPercent(double percentage)
    {
        if (percentage > 30)
        {
            percentage = 30;
            Console.WriteLine("Скидка ограничена 30% для товаров");
        }
        base.ApplyDiscountPercent(percentage);
    }
}

public class ServiceInvoice : Invoice
{
    public DateTime ServiceDate { get; set; }

    private string serviceProvider;
    protected string serviceType;
    internal string serviceLocation;
    public bool isRecurring;

    private string serviceStatus;
    protected string serviceCategory;
    internal int estimatedHours;
    public bool requiresApproval;

    public event Action<string, string> ServiceStatusChanged;

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate)
        : base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
        
        serviceProvider = "Исполнитель";
        serviceType = "Консультация";
        serviceLocation = "Офис";
        isRecurring = false;
        invoiceType = "Услуги";

        serviceStatus = "Запланирована";
        serviceCategory = "Общие услуги";
        estimatedHours = 1;
        requiresApproval = false;
    }

    public void UpdateServiceStatus(string status)
    {
        serviceStatus = status;
        Console.WriteLine($"Статус услуги: {status}");
        ServiceStatusChanged?.Invoke(InvoiceNumber, status);
    }

    public void SetServiceCategory(string category)
    {
        serviceCategory = category;
        Console.WriteLine($"Категория услуги: {category}");
    }

    public void SetEstimatedHours(int hours)
    {
        estimatedHours = hours;
        Console.WriteLine($"Оценка времени: {hours} часов");
    }

    public void RequireApproval(bool require)
    {
        requiresApproval = require;
        if (require)
        {
            Console.WriteLine("Требуется подтверждение услуги");
        }
    }

    public string GetServiceProvider() => serviceProvider;
    
    public void SetRecurring(bool recurring)
    {
        isRecurring = recurring;
        Console.WriteLine(recurring ? "Услуга регулярная" : "Услуга разовая");
    }
    
    public void SetRecurring(bool recurring, int frequencyDays)
    {
        isRecurring = recurring;
        Console.WriteLine($"Услуга регулярная, период: {frequencyDays} дней");
    }

    public void RescheduleService(DateTime newDate)
    {
        ServiceDate = newDate;
        Console.WriteLine($"Услуга перенесена: {newDate:dd.MM.yyyy}");
    }
    
    public void RescheduleService(DateTime newDate, string reason)
    {
        ServiceDate = newDate;
        Console.WriteLine($"Услуга перенесена: {newDate:dd.MM.yyyy}. Причина: {reason}");
    }

    public void GenerateServiceReport()
    {
        Console.WriteLine($"Отчет для {serviceProvider}");
    }

    public override void RemoveLine(LineItem lineItem)
    {
        base.RemoveLine(lineItem);
        Console.WriteLine($"Услуга '{lineItem.Description}' удалена. Причина: отмена");
    }

    public override void ApplyDiscount(decimal discount)
    {
        decimal maxDiscount = TotalAmount * 0.2m;
        decimal actualDiscount = discount > maxDiscount ? maxDiscount : discount;
        base.ApplyDiscount(actualDiscount);
        Console.WriteLine($"Скидка на услуги: {actualDiscount} руб. (макс 20%)");
    }

    public override void ApplyDiscountPercent(double percentage)
    {
        if (percentage > 20)
        {
            percentage = 20;
            Console.WriteLine("Скидка ограничена 20% для услуг");
        }
        base.ApplyDiscountPercent(percentage);
    }

    public override void PrintInfo()
    {
        base.PrintInfo();
        Console.WriteLine($"Услуга: {ServiceDate:dd.MM.yyyy}");
        Console.WriteLine($"Исполнитель: {serviceProvider}");
        Console.WriteLine($"Статус услуги: {serviceStatus}");
        Console.WriteLine($"Категория: {serviceCategory}");
        Console.WriteLine($"Оценка времени: {estimatedHours} часов");
    }
}

public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; set; }
    private decimal returnAmount;

    private string combinedType;
    protected int warrantyMonths;
    internal string returnPolicy;
    public bool hasInstallation;

    private string combinationType;
    protected decimal installationCost;
    internal bool hasWarrantyExtension;
    public string projectManager;

    public event PaymentProcessedHandler PaymentProcessed;

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed)
        : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
        returnAmount = 0;
        
        combinedType = "Товары+Услуги";
        warrantyMonths = 12;
        returnPolicy = "Обычная";
        hasInstallation = false;
        invoiceType = "Комбинированная";

        combinationType = "Стандартная";
        installationCost = 0;
        hasWarrantyExtension = false;
        projectManager = "Менеджер";
    }

    public void SetCombinationType(string type)
    {
        combinationType = type;
        Console.WriteLine($"Тип комбинации: {type}");
    }

    public void SetProjectManager(string manager)
    {
        projectManager = manager;
        Console.WriteLine($"Менеджер проекта: {manager}");
    }

    public void ExtendWarranty(bool extend)
    {
        hasWarrantyExtension = extend;
        if (extend)
        {
            warrantyMonths += 12;
            Console.WriteLine("Гарантия расширена на 12 месяцев");
        }
    }

    public void ProcessPayment(decimal amount)
    {
        Console.WriteLine($"Обработка платежа: {amount} руб.");
        PaymentProcessed?.Invoke(InvoiceNumber, amount);
    }

    public string GetCombinedType() => combinedType;
    
    public void AddWarranty(int months)
    {
        warrantyMonths = months;
        Console.WriteLine($"Гарантия: {months} месяцев");
    }
    
    public void AddWarranty(int months, string warrantyType)
    {
        warrantyMonths = months;
        Console.WriteLine($"Гарантия {warrantyType}: {months} месяцев");
    }

    public void IncludeInstallation()
    {
        hasInstallation = true;
        Console.WriteLine("Установка включена");
    }
    
    public void IncludeInstallation(decimal installationCost)
    {
        hasInstallation = true;
        this.installationCost = installationCost;
        Console.WriteLine($"Установка: {installationCost} руб.");
    }

    public void ProcessReturnRequest(decimal amount)
    {
        if (ReturnAllowed)
        {
            returnAmount = amount;
            Console.WriteLine($"Возврат: {amount} руб.");
        }
    }

    public override void CalculateTotal()
    {
        decimal installationCost = hasInstallation ? this.installationCost : 0;
        TotalAmount = LineItems.Sum(item => item.Amount) - returnAmount + installationCost;
        Console.WriteLine($"Итог комбинированной: {TotalAmount} руб.");
    }

    public void CalculateTotal(bool includeTax)
    {
        CalculateTotal();
        if (includeTax)
        {
            decimal tax = TotalAmount * 0.2m;
            TotalAmount += tax;
            Console.WriteLine($"Сумма с НДС: {TotalAmount} руб.");
        }
    }

    public override void PrintInfo()
    {
        base.PrintInfo();
        Console.WriteLine($"Возврат: {(ReturnAllowed ? "Да" : "Нет")}");
        if (ReturnAllowed && returnAmount > 0)
        {
            Console.WriteLine($"Сумма возврата: {returnAmount} руб.");
        }
        Console.WriteLine($"Гарантия: {warrantyMonths} месяцев");
        Console.WriteLine($"Тип комбинации: {combinationType}");
        Console.WriteLine($"Менеджер проекта: {projectManager}");
    }
}

public interface IInvoiceValidator
{
    bool Validate();
    string[] GetValidationErrors();
}

public interface IInvoiceNotifier
{
    void SendNotification(string message);
}

public interface IInvoiceTaxCalculator
{
    decimal CalculateTax();
}

public interface ILogger
{
    void Log(string message);
    void LogError(string error);
}

public interface IPaymentProcessor
{
    bool ProcessPayment(decimal amount, string invoiceNumber);
}

public interface IEmailService
{
    void SendEmail(string to, string subject, string body);
}

public class ConsoleLogger : ILogger
{
    public void Log(string message)
    {
        Console.WriteLine($"[INFO] {message}");
    }

    public void LogError(string error)
    {
        Console.WriteLine($"[ERROR] {error}");
    }
}

public class PaymentProcessor : IPaymentProcessor
{
    private readonly ILogger _logger;
    public PaymentProcessor(ILogger logger) => _logger = logger;
    
    public bool ProcessPayment(decimal amount, string invoiceNumber)
    {
        _logger.Log($"Платеж {amount} для {invoiceNumber}");
        return amount > 0;
    }
}

public class EmailService : IEmailService
{
    private readonly ILogger _logger;
    public EmailService(ILogger logger) => _logger = logger;
    
    public void SendEmail(string to, string subject, string body)
    {
        _logger.Log($"Email для {to}: {subject}");
        Console.WriteLine($"--- Email ---\nКому: {to}\nТема: {subject}\n{body}\n-----------");
    }
}

public class DependencyContainer
{
    private Dictionary<Type, object> _dependencies = new Dictionary<Type, object>();
    public void Register<T>(T implementation) => _dependencies[typeof(T)] = implementation;
    public T Resolve<T>() => (T)_dependencies[typeof(T)];
}

public class EnhancedInvoice : IInvoiceValidator, IInvoiceNotifier, IInvoiceTaxCalculator
{
    private Invoice _invoice;
    private IEmailService _emailService;
    private ILogger _logger;
    
    public string ValidationStatus { get; private set; }
    public DateTime? LastNotificationDate { get; private set; }
    public decimal TaxRate { get; set; } = 0.2m;
    public bool IsArchived { get; private set; }
    
    public EnhancedInvoice(Invoice invoice, IEmailService emailService, ILogger logger)
    {
        _invoice = invoice;
        _emailService = emailService;
        _logger = logger;
    }
    
    public void Archive()
    {
        IsArchived = true;
        _logger.Log($"Архив: {_invoice.InvoiceNumber}");
    }
    
    public void Restore()
    {
        IsArchived = false;
        _logger.Log($"Восстановлена: {_invoice.InvoiceNumber}");
    }
    
    public string GetEnhancedInfo()
    {
        return $"Улучшенная: {_invoice.InvoiceNumber}";
    }
    
    public void PrintTaxInfo()
    {
        decimal tax = ((IInvoiceTaxCalculator)this).CalculateTax();
        Console.WriteLine($"Налог: {tax} руб.");
    }
    
    bool IInvoiceValidator.Validate()
    {
        ValidationStatus = "OK";
        _logger.Log($"Проверка: {_invoice.InvoiceNumber}");
        return true;
    }
    
    string[] IInvoiceValidator.GetValidationErrors()
    {
        return new string[0];
    }
    
    void IInvoiceNotifier.SendNotification(string message)
    {
        LastNotificationDate = DateTime.Now;
        _emailService.SendEmail("accounting@company.com", $"Фактура {_invoice.InvoiceNumber}", message);
    }
    
    decimal IInvoiceTaxCalculator.CalculateTax()
    {
        var totalAmount = (decimal)_invoice.GetType().GetProperty("TotalAmount").GetValue(_invoice);
        return totalAmount * TaxRate;
    }
}

public class InvoiceService<T> where T : Invoice
{
    private ILogger _logger;
    private IEmailService _emailService;
    private List<T> _invoices = new List<T>();
    
    public InvoiceService(ILogger logger, IEmailService emailService)
    {
        _logger = logger;
        _emailService = emailService;
    }
    
    public void AddInvoice(T invoice)
    {
        _invoices.Add(invoice);
        _logger.Log($"Добавлена: {invoice.InvoiceNumber}");
    }
    
    public void ProcessAllInvoices()
    {
        foreach (var invoice in _invoices)
        {
            var enhanced = new EnhancedInvoice(invoice, _emailService, _logger);
            ((IInvoiceValidator)enhanced).Validate();
            ((IInvoiceNotifier)enhanced).SendNotification("Обработана");
            enhanced.PrintTaxInfo();
        }
    }
}


    
    {
        Console.WriteLine("=== Система фактур ===\n");

        var invoiceCollection = new InvoiceCollection<Invoice>();
        
        invoiceCollection.CollectionChanged += (invoiceNumber, action) => 
        {
            Console.WriteLine($"[КОЛЛЕКЦИЯ] {action}: {invoiceNumber}");
        };

        GoodsInvoice goods = new GoodsInvoice("G-001", new DateTime(2024, 12, 21), new DateTime(2024, 12, 25));
        
        goods.InvoiceCreated += (sender, e) => Console.WriteLine($"[СОБЫТИЕ] {e.Message}: {e.InvoiceNumber}");
        goods.DeliveryScheduled += (invNum, date) => Console.WriteLine($"[ДОСТАВКА] Запланирована для {invNum} на {date:dd.MM.yyyy}");

        goods.AddLine(new LineItem("Ноутбук", 50000, 1));
        goods.AddLine("Мышь", 1000, 2);
        
        goods.SetCustomer("ООО ТехноПрофи", "ул. Пушкина, 25");
        goods.ChangeStatus("Подтверждена");
        goods.AddShippingCost(1500);
        goods.SetTrackingNumber("TRK-123456789");
        goods.UpdateDeliveryStatus("В пути");
        
        goods.PrintInfo();
        invoiceCollection.AddInvoice(goods);
        Console.WriteLine();

        ServiceInvoice service = new ServiceInvoice("S-001", new DateTime(2024, 12, 21), new DateTime(2024, 12, 22));
        
        service.ServiceStatusChanged += (invNum, status) => Console.WriteLine($"[УСЛУГА] Статус изменен для {invNum}: {status}");

        service.AddLine("Консультация", 5000, 2);
        service.SetRecurring(true, 30);
        
        service.SetServiceCategory("IT консультации");
        service.SetEstimatedHours(4);
        service.UpdateServiceStatus("В процессе");
        service.RequireApproval(true);
        
        service.PrintInfo(true);
        invoiceCollection.AddInvoice(service);
        Console.WriteLine();

        CombinedInvoice combined = new CombinedInvoice("C-001", new DateTime(2024, 12, 21), true);
        
        combined.PaymentProcessed += (invNum, amount) => Console.WriteLine($"[ПЛАТЕЖ] Обработан для {invNum}: {amount} руб.");

        combined.AddLine("Товар", 10000, 1);
        combined.AddWarranty(24, "Расширенная");
        combined.IncludeInstallation(3000);
        
        combined.SetCombinationType("Товар с установкой");
        combined.SetProjectManager("Иванов И.И.");
        combined.ExtendWarranty(true);
        combined.ProcessPayment(15000);
        
        combined.CalculateTotal(true);
        combined.PrintInfo();
        invoiceCollection.AddInvoice(combined);
        Console.WriteLine();

        Console.WriteLine("=== Полиморфизм ===");
        goods.ScheduleDelivery(DateTime.Now.AddDays(5), "Экспресс");
        goods.MarkAsFragile("Высокий");
        service.RescheduleService(DateTime.Now.AddDays(3), "Занятость");
        goods.ApplyDiscount(10000m);
        service.ApplyDiscount(5000m);
        combined.ApplyDiscountPercent(10.0);
        combined.ProcessReturnRequest(2000);
        
        Console.WriteLine("\n=== Обновленные данные ===");
        goods.PrintInfo();
        service.PrintInfo();
        combined.PrintInfo();

        Console.WriteLine("\n" + new string('=', 50));
        invoiceCollection.PrintAllInvoices();

        Console.WriteLine("\n=== Смена валюты ===");
        goods.ChangeCurrency("USD");
        goods.ChangeCurrency("EUR", 0.9m);

        Console.WriteLine("\n" + new string('=', 60));
        Console.WriteLine("=== Дополнительно ===");
        
        var container = new DependencyContainer();
        container.Register<ILogger>(new ConsoleLogger());
        container.Register<IEmailService>(new EmailService(container.Resolve<ILogger>()));
        container.Register<IPaymentProcessor>(new PaymentProcessor(container.Resolve<ILogger>()));
        
        var logger = container.Resolve<ILogger>();
        var emailService = container.Resolve<IEmailService>();
        var paymentProcessor = container.Resolve<IPaymentProcessor>();
        
        var goodsService = new InvoiceService<GoodsInvoice>(logger, emailService);
        var serviceService = new InvoiceService<ServiceInvoice>(logger, emailService);
        
        var testGoods = new GoodsInvoice("EXT-001", DateTime.Now, DateTime.Now.AddDays(5));
        testGoods.AddLine("Тест товар", 5000, 2);
        
        var testService = new ServiceInvoice("EXT-002", DateTime.Now, DateTime.Now.AddDays(2));
        testService.AddLine("Тест услуга", 3000, 1);
        
        goodsService.AddInvoice(testGoods);
        serviceService.AddInvoice(testService);
        
        Console.WriteLine("\n--- Сервисы ---");
        goodsService.ProcessAllInvoices();
        serviceService.ProcessAllInvoices();
        
        Console.WriteLine("\n--- Интерфейсы ---");
        var enhanced = new EnhancedInvoice(testGoods, emailService, logger);
        IInvoiceValidator validator = enhanced;
        validator.Validate();
        IInvoiceNotifier notifier = enhanced;
        notifier.SendNotification("Тест");
        IInvoiceTaxCalculator calculator = enhanced;
        decimal tax = calculator.CalculateTax();
        Console.WriteLine($"Налог: {tax} руб.");
        
        enhanced.Archive();
        enhanced.Restore();
        Console.WriteLine(enhanced.GetEnhancedInfo());
        
        Console.WriteLine("\n--- Платежи ---");
        bool paymentResult = paymentProcessor.ProcessPayment(10000, "TEST-001");
        Console.WriteLine($"Платеж: {(paymentResult ? "Успех" : "Ошибка")}");
        
        Console.WriteLine("\n=== Работа с коллекцией ===");
        Console.WriteLine($"Всего фактур: {invoiceCollection.GetInvoiceCount()}");
        Console.WriteLine($"Общая сумма: {invoiceCollection.GetTotalAmount()} руб.");
        
        var recentInvoices = invoiceCollection.GetInvoicesByDateRange(
            new DateTime(2024, 12, 1), 
            new DateTime(2024, 12, 31));
        
        Console.WriteLine($"Фактур за декабрь 2024: {recentInvoices.Count()}");

        Console.WriteLine("\n=== Готово ===");
    }


=== Система фактур ===

Товар 'Ноутбук' поставка 25.12.2024
Товар 'Мышь' поставка 25.12.2024
Клиент: ООО ТехноПрофи, Адрес: ул. Пушкина, 25
Статус изменен: Черновик -> Подтверждена
Стоимость доставки: 1500 руб.
Трек номер: TRK-123456789
Статус доставки: В пути
Фактура #G-001
Дата: 21.12.2024
Сумма: 53500 RUB
Поставка: 25.12.2024
Поставщик: Поставщик
Статус доставки: В пути
Трек номер: TRK-123456789
Фактура G-001 добавлена
[КОЛЛЕКЦИЯ] Добавлена в коллекцию: G-001

Добавлено: Консультация
Услуга регулярная, период: 30 дней
Категория услуги: IT консультации
Оценка времени: 4 часов
Статус услуги: В процессе
[УСЛУГА] Статус изменен для S-001: В процессе
Требуется подтверждение услуги
--- Фактура #S-001 ---
Дата: 21.12.2024
Тип: Услуги
Сумма: 10000 RUB
Оплата: Безналичный
Условия: 30 дней
Позиций: 1
Статус: Черновик
Клиент: Клиент
Оплата до: 20.01.2025
Фактура S-001 добавлена
[КОЛЛЕКЦИЯ] Добавлена в коллекцию: S-001

Итог комбинированной: 10000 руб.
Добавлено: Товар
Гарантия Расширенная: 24 